# Projet Final Apache Spark

**Nom Etudiant : BASSENE  

**Prenom Etudiant: Khalifa Ababacar  

**Classe : Intelligence Artificielle


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [5]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.functions._



import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.functions._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [6]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [7]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [8]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [9]:
fireDF.cache()

res2: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [10]:
fireDF.count()

res3: Long = 175296


In [11]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [20]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [12]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [15]:
// Reponse 1
/*
Ecrire ici votre code
*/
import org.apache.spark.sql.functions.countDistinct

println ("Le nombre de types d'appels passées est de : " 
  + fireDF
  .select("CallType")
  .distinct()
  .where($"CallType" =!= "")
  .count()
  + " appel(s) "       
)

Le nombre de types d'appels passées est de : 30 appel(s) 


import org.apache.spark.sql.functions.countDistinct


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [16]:
// Reponse 2
val TypeAppels = fireDF
.select("CallType")
  .distinct()
TypeAppels.show()

+--------------------+
|            CallType|
+--------------------+
|Elevator / Escala...|
|         Marine Fire|
|  Aircraft Emergency|
|Confined Space / ...|
|      Administrative|
|              Alarms|
|Odor (Strange / U...|
|Citizen Assist / ...|
|              HazMat|
|Watercraft in Dis...|
|           Explosion|
|           Oil Spill|
|        Vehicle Fire|
|  Suspicious Package|
|Extrication / Ent...|
|               Other|
|        Outside Fire|
|   Traffic Collision|
|       Assist Police|
|Gas Leak (Natural...|
+--------------------+
only showing top 20 rows



TypeAppels: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string]


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

**Commentaire**

A votre code pour renommer le le champ delay, j'ai ajouté une condition permettant de ne sélectionner que les Delay qui sont supérieur à 5. Vu que cette fois ci on nous a demandé d'afficher tous les appels, j'ai pas mis d'attributs  à la fonction show pour l'affichage.

In [17]:
// Reponse 3
// - 1 & 2 
val newFireDF = fireDF.
    withColumnRenamed("Delay", "ResponseDelayedinMins")
    .where($"ResponseDelayedinMins" > "5")

newFireDF.show()

+----------+------+--------------+--------------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------------------+
|CallNumber|UnitID|IncidentNumber|            CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|ResponseDelayedinMins|
+----------+------+--------------+--------------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-------

newFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [20]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res13: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

**Commentaires**

Pour afficher les types d'appels les plus courants, je l'ai d'abord regroupé en éléments semblable avant de les compter et de les affichés tout en prenant soisn de ne sélectionner que ceux qui ont un nombre supérieur à 100

In [21]:
//Reponse 4
fireTSDF
    .groupBy("CallType")
    .count().filter("`count` >= 10")
    .sort(desc("count"))
    .show(false)

+-------------------------------+-----+
|CallType                       |count|
+-------------------------------+-----+
|Medical Incident               |18981|
|Structure Fire                 |2424 |
|Alarms                         |1797 |
|Traffic Collision              |1193 |
|Citizen Assist / Service Call  |803  |
|Other                          |585  |
|Water Rescue                   |332  |
|Outside Fire                   |305  |
|Gas Leak (Natural and LP Gases)|167  |
|Odor (Strange / Unknown)       |164  |
|Vehicle Fire                   |123  |
|Electrical Hazard              |121  |
|Elevator / Escalator Rescue    |94   |
|Smoke Investigation (Outside)  |94   |
|HazMat                         |76   |
|Fuel Spill                     |64   |
|Industrial Accidents           |33   |
|Train / Rail Incident          |32   |
|Watercraft in Distress         |21   |
|Explosion                      |16   |
+-------------------------------+-----+
only showing top 20 rows



### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

Ici, j'ai repris le même code que celui de la question 4 le nombre de boites postaux est 

In [22]:
//Reponse 5-a
fireTSDF
    .groupBy("Zipcode")
    .count()
    .filter("`count` >= 10")
    .sort(desc("count"))
    .show(false)

+-------+-----+
|Zipcode|count|
+-------+-----+
|94102  |3285 |
|94103  |3262 |
|94110  |2220 |
|94109  |2189 |
|94124  |1455 |
|94112  |1310 |
|94107  |1131 |
|94115  |1098 |
|94122  |1042 |
|94133  |1005 |
|94134  |877  |
|94118  |795  |
|94117  |790  |
|94121  |788  |
|94132  |768  |
|94114  |751  |
|94105  |687  |
|94116  |630  |
|94108  |629  |
|94131  |560  |
+-------+-----+
only showing top 20 rows



### Question 5-b
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [26]:
//Reponse 5-b
val fewF = fireDF
  .select("Address") 
  .where($"Zipcode" === "94102", $"Zipcode" === "94103")



<console>: 47: error: overloaded method value where with alternatives:

### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [27]:
//Reponse 6
//le nombre total d'appels
println("Le nombre d'appels est de : "+ fireDF.count())

fireDF.groupBy("Zipcode")
    .agg(
      min("Delay").as("Minimum"),
      avg("Delay").as("Moyenne"),
      max("Delay").as("Maximum"))
    .show(false)

Le nombre d'appels est de : 175296
+-------+-----------+------------------+----------+
|Zipcode|Minimum    |Moyenne           |Maximum   |
+-------+-----------+------------------+----------+
|94109  |0.083333336|3.7603318356150024|336.33334 |
|94115  |0.016666668|3.7328533033765967|87.21667  |
|94112  |0.2        |3.744305901877912 |83.48333  |
|94127  |0.21666667 |3.9756955544819292|124.05    |
|94108  |0.16666667 |3.727893405607614 |54.583332 |
|94121  |0.31666666 |3.9780241508711573|106.333336|
|94105  |0.26666668 |4.152647933306223 |628.61664 |
|null   |0.016666668|4.298591555240498 |30.65     |
|94131  |0.25       |4.083008859974271 |114.7     |
|94116  |0.26666668 |3.828862613472901 |199.45    |
|94134  |0.1        |4.10027284515781  |405.7     |
|94124  |0.11666667 |4.142393898997325 |734.05    |
|94102  |0.06666667 |3.7218963706603225|340.48334 |
|94114  |0.083333336|3.6728115956204523|88.71667  |
|94107  |0.15       |3.8436920714793126|239.95    |
|94111  |0.23333333 |3.850846

### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [29]:
//Reponse 7-a
fireTSDF
.select( year(col("CallDate")))
.distinct()

org.apache.spark.sql.AnalysisException:  cannot resolve '`CallDate`' given input columns: [ALSUnit, Address, AvailableDtTS, Battalion, Box, CallFinalDisposition, CallNumber, CallType, CallTypeGroup, City, FinalPriority, FirePreventionDistrict, IncidentDate, IncidentNumber, Location, Neighborhood, NumAlarms, OnWatchDate, OriginalPriority, Priority, ResponseDelayedinMins, RowID, StationArea, SupervisorDistrict, UnitID, UnitSequenceInCallDispatch, UnitType, Zipcode];;

### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [31]:
//Reponse 7-b



res21: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string]


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [20]:
//Reponse 8
/*
Ecrire ici votre code
*/

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [21]:
//Reponse 9
/*
Ecrire ici votre code
*/

### Question 10
**Comment relire les données stockée en format Parquet?**

In [22]:
//Reponse 10
/*
Ecrire ici votre code
*/

## FIN